In [0]:
#Getting all libraries necessary for now!
import pandas as pd
import numpy as np
from datetime import timedelta

import matplotlib.pyplot as plt
import seaborn as sns

import datetime

import warnings
warnings.filterwarnings('ignore')

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Read all these to pandas!
train = pd.read_csv("/content/drive/My Drive/LTFS/train_fwYjLYX.csv")
test = pd.read_csv("/content/drive/My Drive/LTFS/test_1eLl9Yf.csv")
submission = pd.read_csv("/content/drive/My Drive/LTFS/sample_submission_IIzFVsf.csv")

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [4]:
print(train.shape, test.shape)

(80402, 6) (180, 3)


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80402 entries, 0 to 80401
Data columns (total 6 columns):
application_date    80402 non-null object
segment             80402 non-null int64
branch_id           66898 non-null float64
state               80402 non-null object
zone                66898 non-null object
case_count          80402 non-null float64
dtypes: float64(2), int64(1), object(3)
memory usage: 3.7+ MB


In [0]:
train['application_date'] = pd.to_datetime(train['application_date'])
test['application_date'] = pd.to_datetime(test['application_date'])

In [7]:
import holidays
train = train.sort_values('application_date')
train.head()

,application_date,segment,branch_id,state,zone,case_count
0,2017-04-01,1,1.0,WEST BENGAL,EAST,40.0
20150,2017-04-01,1,62.0,PUNJAB,NORTH,2.0
20956,2017-04-01,1,63.0,JHARKHAND,SOUTH,6.0
21762,2017-04-01,1,64.0,KARNATAKA,SOUTH,0.0
22568,2017-04-01,1,65.0,ASSAM,EAST,12.0


In [8]:
train = train.reset_index(drop=True)
train.head(100)

,application_date,segment,branch_id,state,zone,case_count
0,2017-04-01,1,1.0,WEST BENGAL,EAST,40.0
1,2017-04-01,1,62.0,PUNJAB,NORTH,2.0
2,2017-04-01,1,63.0,JHARKHAND,SOUTH,6.0
3,2017-04-01,1,64.0,KARNATAKA,SOUTH,0.0
4,2017-04-01,1,65.0,ASSAM,EAST,12.0
...,...,...,...,...,...,...
95,2017-04-01,1,250.0,GUJARAT,WEST,0.0
96,2017-04-01,2,NaN,KARNATAKA,NaN,165.0
97,2017-04-01,1,3.0,DELHI,NORTH,58.0
98,2017-04-01,2,NaN,HARYANA,NaN,0.0


In [9]:
#Splitting the dataframe into Segment 1 and 2
train1 = train[train['segment']==1]
train2 = train[train['segment']==2]
train1 = train1.reset_index(drop=True)
train2 = train2.reset_index(drop=True)
print(train1.head())
print(train2.head())

  application_date  segment  branch_id        state   zone  case_count
0       2017-04-01        1        1.0  WEST BENGAL   EAST        40.0
1       2017-04-01        1       62.0       PUNJAB  NORTH         2.0
2       2017-04-01        1       63.0    JHARKHAND  SOUTH         6.0
3       2017-04-01        1       64.0    KARNATAKA  SOUTH         0.0
4       2017-04-01        1       65.0        ASSAM   EAST        12.0
  application_date  segment  branch_id          state zone  case_count
0       2017-04-01        2        NaN     TAMIL NADU  NaN       458.0
1       2017-04-01        2        NaN         PUNJAB  NaN         0.0
2       2017-04-01        2        NaN        TRIPURA  NaN         0.0
3       2017-04-01        2        NaN    WEST BENGAL  NaN        43.0
4       2017-04-01        2        NaN  UTTAR PRADESH  NaN        27.0


In [10]:
date_train1 = train1.drop(['branch_id','state','zone'], axis=1)
date_train2 = train2.drop(['branch_id','state','zone'], axis=1)
print(date_train1.head())
print(date_train2.head())

  application_date  segment  case_count
0       2017-04-01        1        40.0
1       2017-04-01        1         2.0
2       2017-04-01        1         6.0
3       2017-04-01        1         0.0
4       2017-04-01        1        12.0
  application_date  segment  case_count
0       2017-04-01        2       458.0
1       2017-04-01        2         0.0
2       2017-04-01        2         0.0
3       2017-04-01        2        43.0
4       2017-04-01        2        27.0


In [11]:
date_train1 = date_train1.groupby('application_date',as_index= False).agg({'case_count':'sum'})
date_train1['segment'] = 1
date_train1 = date_train1[['application_date','segment','case_count']]
date_train1.head()

,application_date,segment,case_count
0,2017-04-01,1,299.0
1,2017-04-03,1,42.0
2,2017-04-04,1,23.0
3,2017-04-05,1,1530.0
4,2017-04-07,1,1341.0


In [12]:
date_train2 = date_train2.groupby('application_date',as_index= False).agg({'case_count':'sum'})
date_train2['segment'] = 2
date_train2 = date_train2[['application_date','segment','case_count']]
date_train2.head()

,application_date,segment,case_count
0,2017-04-01,2,897.0
1,2017-04-02,2,605.0
2,2017-04-03,2,2016.0
3,2017-04-04,2,2245.0
4,2017-04-05,2,2415.0


In [13]:
date_train1.describe() # There are outliers, as expected!

,segment,case_count
count,806.0,806.000000
mean,1.0,2714.689826
std,0.0,2042.418090
min,1.0,1.000000
25%,1.0,1633.250000
50%,1.0,2602.000000
75%,1.0,3333.000000
max,1.0,21885.000000


In [14]:
date_train2.describe()

,segment,case_count
count,844.0,844.000000
mean,2.0,15077.098341
std,0.0,10188.900682
min,2.0,206.000000
25%,2.0,5946.000000
50%,2.0,12679.000000
75%,2.0,24217.250000
max,2.0,43340.000000


In [0]:
test1 = test[test['segment']==1]
test2 = test[test['segment']==2]

In [16]:
test1.drop('id', axis=1, inplace = True)
test2.drop('id', axis=1, inplace = True)
print(test1.head())
print(test2.head())

  application_date  segment
0       2019-07-06        1
1       2019-07-07        1
2       2019-07-08        1
3       2019-07-09        1
4       2019-07-10        1
   application_date  segment
87       2019-07-24        2
88       2019-07-25        2
89       2019-07-26        2
90       2019-07-27        2
91       2019-07-28        2


In [17]:
print(date_train1.shape, date_train2.shape, test1.shape, test2.shape)

(806, 3) (844, 3) (87, 2) (93, 2)


In [18]:
df1 = date_train1.append(test1, ignore_index = True)
df2 = date_train2.append(test2, ignore_index = True)
print(df1.shape)
print(df2.shape)
print(df1.head())
print(df2.head())

(893, 3)
(937, 3)
  application_date  case_count  segment
0       2017-04-01       299.0        1
1       2017-04-03        42.0        1
2       2017-04-04        23.0        1
3       2017-04-05      1530.0        1
4       2017-04-07      1341.0        1
  application_date  case_count  segment
0       2017-04-01       897.0        2
1       2017-04-02       605.0        2
2       2017-04-03      2016.0        2
3       2017-04-04      2245.0        2
4       2017-04-05      2415.0        2


In [0]:
from fastai.tabular import add_datepart

add_datepart(df1, 'application_date', drop=False)

df1.drop('application_Elapsed', axis=1, inplace=True)

In [20]:
df1.shape

(893, 15)

In [0]:
from fastai.tabular import add_datepart

add_datepart(df2, 'application_date', drop=False)

df2.drop('application_Elapsed', axis=1, inplace=True)

In [22]:
import holidays
ind_holidays = []

for date in holidays.India(years =[2017,2018,2019]).items():
  ind_holidays.append(str(date[0]))

print(ind_holidays, end=',')
#The below list does not make any sense! There is no festival holidays available!

['2017-01-14', '2017-01-26', '2017-08-15', '2017-10-02', '2017-05-01', '2017-12-25', '2018-01-14', '2018-01-26', '2018-08-15', '2018-10-02', '2018-05-01', '2018-12-25', '2019-01-14', '2019-01-26', '2019-08-15', '2019-10-02', '2019-05-01', '2019-12-25'],

In [23]:
# Refering calendarlabs.com
#['2017-01-01','2017-03-13','2017-03-28','2017-04-05','2017-04-14','2017-06-26','2017-08-07','2017-08-14','2017-08-25',
#'2017-09-04','2017-09-30','2017-10-19','2018-01-01','2018-03-18','2018-03-25','2018-03-30','2018-04-14','2018-06-16','2018-08-25',
#'2018-08-26','2018-10-19','2018-11-07','2019-01-01','2019-03-21','2019-04-06','2019-04-13','2019-04-19','2019-06-05','2019-08-24',
#'2019-09-02','2019-09-11','2019-10-08','2019-10-27'])
ind_holidays.append('2017-01-01')
ind_holidays.append('2017-03-13')
ind_holidays.append('2017-03-28')
ind_holidays.append('2017-04-05')
ind_holidays.append('2017-04-14')
ind_holidays.append('2017-06-26')
ind_holidays.append('2017-08-07')
ind_holidays.append('2017-08-14')
ind_holidays.append('2017-08-25')
ind_holidays.append('2017-09-04')
ind_holidays.append('2017-09-30')
ind_holidays.append('2017-10-19')
ind_holidays.append('2018-01-01')
ind_holidays.append('2018-03-18')
ind_holidays.append('2018-03-25')
ind_holidays.append('2018-03-30')
ind_holidays.append('2018-04-14')
ind_holidays.append('2018-06-16')
ind_holidays.append('2018-08-25')
ind_holidays.append('2018-08-26')
ind_holidays.append('2018-10-19')
ind_holidays.append('2018-11-07')
ind_holidays.append('2019-01-01')
ind_holidays.append('2019-03-21')
ind_holidays.append('2019-04-06')
ind_holidays.append('2019-04-13')
ind_holidays.append('2019-04-19')
ind_holidays.append('2019-06-05')
ind_holidays.append('2019-08-24')
ind_holidays.append('2019-09-02')
ind_holidays.append('2019-09-11')
ind_holidays.append('2019-10-08')
ind_holidays.append('2019-10-27')
ind_holidays

['2017-01-14',
 '2017-01-26',
 '2017-08-15',
 '2017-10-02',
 '2017-05-01',
 '2017-12-25',
 '2018-01-14',
 '2018-01-26',
 '2018-08-15',
 '2018-10-02',
 '2018-05-01',
 '2018-12-25',
 '2019-01-14',
 '2019-01-26',
 '2019-08-15',
 '2019-10-02',
 '2019-05-01',
 '2019-12-25',
 '2017-01-01',
 '2017-03-13',
 '2017-03-28',
 '2017-04-05',
 '2017-04-14',
 '2017-06-26',
 '2017-08-07',
 '2017-08-14',
 '2017-08-25',
 '2017-09-04',
 '2017-09-30',
 '2017-10-19',
 '2018-01-01',
 '2018-03-18',
 '2018-03-25',
 '2018-03-30',
 '2018-04-14',
 '2018-06-16',
 '2018-08-25',
 '2018-08-26',
 '2018-10-19',
 '2018-11-07',
 '2019-01-01',
 '2019-03-21',
 '2019-04-06',
 '2019-04-13',
 '2019-04-19',
 '2019-06-05',
 '2019-08-24',
 '2019-09-02',
 '2019-09-11',
 '2019-10-08',
 '2019-10-27']

In [24]:
start_date = datetime.datetime(year=2017, month=1, day=1)
end_date = datetime.datetime(year=2019, month=12, day=31)

pd.date_range(start_date, end_date)

DatetimeIndex(['2017-01-01', '2017-01-02', '2017-01-03', '2017-01-04',
               '2017-01-05', '2017-01-06', '2017-01-07', '2017-01-08',
               '2017-01-09', '2017-01-10',
               ...
               '2019-12-22', '2019-12-23', '2019-12-24', '2019-12-25',
               '2019-12-26', '2019-12-27', '2019-12-28', '2019-12-29',
               '2019-12-30', '2019-12-31'],
              dtype='datetime64[ns]', length=1095, freq='D')

In [25]:
date_range = pd.DataFrame()
date_range['application_date'] = pd.date_range(start_date, end_date)
date_range.head() 

,application_date
0,2017-01-01
1,2017-01-02
2,2017-01-03
3,2017-01-04
4,2017-01-05


In [0]:
date_range['is_Holiday'] = [1 if str(val).split()[0] in ind_holidays else 0 for val in date_range['application_date']]

In [27]:
date_range.head(100)

,application_date,is_Holiday
0,2017-01-01,1
1,2017-01-02,0
2,2017-01-03,0
3,2017-01-04,0
4,2017-01-05,0
...,...,...
95,2017-04-06,0
96,2017-04-07,0
97,2017-04-08,0
98,2017-04-09,0


In [28]:
df1 = df1.merge(date_range, how = 'left', on = ['application_date'])
df2 = df2.merge(date_range, how = 'left', on = ['application_date'])
df1.head()

,application_date,case_count,segment,application_Year,application_Month,application_Week,application_Day,application_Dayofweek,application_Dayofyear,application_Is_month_end,application_Is_month_start,application_Is_quarter_end,application_Is_quarter_start,application_Is_year_end,application_Is_year_start,is_Holiday
0,2017-04-01,299.0,1,2017,4,13,1,5,91,False,True,False,True,False,False,0
1,2017-04-03,42.0,1,2017,4,14,3,0,93,False,False,False,False,False,False,0
2,2017-04-04,23.0,1,2017,4,14,4,1,94,False,False,False,False,False,False,0
3,2017-04-05,1530.0,1,2017,4,14,5,2,95,False,False,False,False,False,False,1
4,2017-04-07,1341.0,1,2017,4,14,7,4,97,False,False,False,False,False,False,0


In [29]:
df2.head()

,application_date,case_count,segment,application_Year,application_Month,application_Week,application_Day,application_Dayofweek,application_Dayofyear,application_Is_month_end,application_Is_month_start,application_Is_quarter_end,application_Is_quarter_start,application_Is_year_end,application_Is_year_start,is_Holiday
0,2017-04-01,897.0,2,2017,4,13,1,5,91,False,True,False,True,False,False,0
1,2017-04-02,605.0,2,2017,4,13,2,6,92,False,False,False,False,False,False,0
2,2017-04-03,2016.0,2,2017,4,14,3,0,93,False,False,False,False,False,False,0
3,2017-04-04,2245.0,2,2017,4,14,4,1,94,False,False,False,False,False,False,0
4,2017-04-05,2415.0,2,2017,4,14,5,2,95,False,False,False,False,False,False,1


In [0]:
#df1['segment'] = df1['segment'].astype('category')
#df2['segment'] = df2['segment'].astype('category')

In [31]:
df1_train=df1[df1['case_count'].isnull()==False].copy()
df1_test=df1[df1['case_count'].isnull()==True].copy()
df2_train=df2[df2['case_count'].isnull()==False].copy()
df2_test=df2[df2['case_count'].isnull()==True].copy()

print(df1_train.shape,df1_test.shape,df2_train.shape,df2_test.shape)


(806, 16) (87, 16) (844, 16) (93, 16)


In [0]:
df1_train = df1_train[df1_train['case_count'] > 15].reset_index(drop=True)
df1_train = df1_train[df1_train['case_count'] <= 8000].reset_index(drop=True)
df2_train = df2_train[df2_train['case_count'] > 15].reset_index(drop=True)
df2_train = df2_train[df2_train['case_count'] < 40000].reset_index(drop=True)

In [0]:
from xgboost import XGBRFRegressor
from sklearn.model_selection import StratifiedKFold,train_test_split
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error

In [34]:
X1,Y1=df1_train.drop(['application_date','case_count'],axis=1),df1_train['case_count']
X1test=df1_test.drop(['application_date','case_count'],axis=1)

X2,Y2=df2_train.drop(['application_date','case_count'],axis=1),df2_train['case_count']
X2test=df2_test.drop(['application_date','case_count'],axis=1)
print(X1.shape,X1test.shape,X2.shape,X2test.shape)
X1_train,X1_val,Y1_train,Y1_val = train_test_split(X1,Y1,test_size=0.25,random_state = 2020)
X2_train,X2_val,Y2_train,Y2_val = train_test_split(X2,Y2,test_size=0.25,random_state = 2020)

(780, 14) (87, 14) (843, 14) (93, 14)


In [35]:
LG = LGBMRegressor(n_estimators=3000)
LG.fit(X1_train, Y1_train)
y_pred = LG.predict(X1_val)
print('MAPE:', mean_absolute_percentage_error(Y1_val, y_pred))

MAPE: 69.87273057754845


In [36]:
LGBMerror1=[]
y_pred_LGBM1=[]
fold=StratifiedKFold(n_splits=3,shuffle=True,random_state=402)
i=1
for train_index, test_index in fold.split(X1,Y1):
    X1_train, X1_test = X1.iloc[train_index], X1.iloc[test_index]
    Y1_train, Y1_test = Y1[train_index], Y1[test_index]
    model=LGBMRegressor(boosting_type='gbdt',n_estimators=3000,learning_rate = 0.01, objective='regression', colsample_bytree=0.3)
    model.fit(X1_train,Y1_train,eval_set=[(X1_test, Y1_test)], early_stopping_rounds=5000, verbose=200)
    preds=model.predict(X1_test)[:]
    print("LGBMerror: ",mean_absolute_percentage_error(Y1_test,preds))
    LGBMerror1.append(mean_absolute_percentage_error(Y1_test,preds))
    p = model.predict(X1test)[:]
    i=i+1
    y_pred_LGBM1.append(p)

Training until validation scores don't improve for 5000 rounds.
[200]	valid_0's l2: 1.01448e+06
[400]	valid_0's l2: 808788
[600]	valid_0's l2: 729457
[800]	valid_0's l2: 700981
[1000]	valid_0's l2: 692184
[1200]	valid_0's l2: 690341
[1400]	valid_0's l2: 687337
[1600]	valid_0's l2: 685411
[1800]	valid_0's l2: 690230
[2000]	valid_0's l2: 694005
[2200]	valid_0's l2: 696500
[2400]	valid_0's l2: 700297
[2600]	valid_0's l2: 704156
[2800]	valid_0's l2: 706429
[3000]	valid_0's l2: 709871
Did not meet early stopping. Best iteration is:
[1577]	valid_0's l2: 685197
LGBMerror:  96.0309188367591
Training until validation scores don't improve for 5000 rounds.
[200]	valid_0's l2: 1.11625e+06
[400]	valid_0's l2: 914228
[600]	valid_0's l2: 833447
[800]	valid_0's l2: 801225
[1000]	valid_0's l2: 785588
[1200]	valid_0's l2: 772458
[1400]	valid_0's l2: 768863
[1600]	valid_0's l2: 769105
[1800]	valid_0's l2: 768023
[2000]	valid_0's l2: 766277
[2200]	valid_0's l2: 765345
[2400]	valid_0's l2: 765187
[2600]	va

In [37]:
LG = LGBMRegressor(n_estimators=3000)
LG.fit(X2_train, Y2_train)
y_pred = LG.predict(X2_val)
print('MAPE:', mean_absolute_percentage_error(Y2_val, y_pred))

MAPE: 24.611561158200754


In [38]:
LGBMerror2=[]
y_pred_LGBM2=[]
fold=StratifiedKFold(n_splits=2,shuffle=True,random_state=402)
i=1
for train_index, test_index in fold.split(X2,Y2):
    X2_train, X2_test = X2.iloc[train_index], X2.iloc[test_index]
    Y2_train, Y2_test = Y2[train_index], Y2[test_index]
    model=LGBMRegressor(boosting_type='gbdt',n_estimators=3000,learning_rate = 0.01, objective='regression', colsample_bytree=0.3)
    model.fit(X2_train,Y2_train,eval_set=[(X2_test, Y2_test)], early_stopping_rounds=5000, verbose=200)
    preds=model.predict(X2_test)[:]
    print("LGBMerror: ",mean_absolute_percentage_error(Y2_test,preds))
    LGBMerror2.append(mean_absolute_percentage_error(Y2_test,preds))
    p = model.predict(X2test)[:]
    i=i+1
    y_pred_LGBM2.append(p)

Training until validation scores don't improve for 5000 rounds.
[200]	valid_0's l2: 5.00634e+07
[400]	valid_0's l2: 3.37295e+07
[600]	valid_0's l2: 2.68169e+07
[800]	valid_0's l2: 2.38974e+07
[1000]	valid_0's l2: 2.2235e+07
[1200]	valid_0's l2: 2.09631e+07
[1400]	valid_0's l2: 2.01409e+07
[1600]	valid_0's l2: 1.96596e+07
[1800]	valid_0's l2: 1.92508e+07
[2000]	valid_0's l2: 1.89545e+07
[2200]	valid_0's l2: 1.87244e+07
[2400]	valid_0's l2: 1.85366e+07
[2600]	valid_0's l2: 1.8355e+07
[2800]	valid_0's l2: 1.81935e+07
[3000]	valid_0's l2: 1.80524e+07
Did not meet early stopping. Best iteration is:
[3000]	valid_0's l2: 1.80524e+07
LGBMerror:  65.46050241853627
Training until validation scores don't improve for 5000 rounds.
[200]	valid_0's l2: 5.2696e+07
[400]	valid_0's l2: 3.57108e+07
[600]	valid_0's l2: 2.77916e+07
[800]	valid_0's l2: 2.42062e+07
[1000]	valid_0's l2: 2.22293e+07
[1200]	valid_0's l2: 2.08393e+07
[1400]	valid_0's l2: 1.99218e+07
[1600]	valid_0's l2: 1.93892e+07
[1800]	valid_

In [0]:
submission1 = submission[submission['segment'] == 1]
submission2 = submission[submission['segment'] == 2]

In [0]:
submission1['case_count'] = np.abs(np.mean(y_pred_LGBM1,0))
submission2['case_count'] = np.abs(np.mean(y_pred_LGBM2,0))

In [41]:
final_submission = pd.concat([submission1,submission2])
final_submission.head(15)

,id,application_date,segment,case_count
0,1,2019-07-06,1,2845.574291
1,2,2019-07-07,1,1701.228087
2,3,2019-07-08,1,3753.261056
3,4,2019-07-09,1,3249.201928
4,5,2019-07-10,1,3375.167693
5,6,2019-07-11,1,3306.694828
6,7,2019-07-12,1,3508.336863
7,8,2019-07-13,1,2966.270847
8,9,2019-07-14,1,1783.247127
9,10,2019-07-15,1,3703.165918


In [42]:
final_submission.to_csv("/content/drive/My Drive/LTFS/final_submission_LGBM.csv", index= False)
final_submission.shape

(180, 4)